# Preparing your data for the model

Now that you have your data, you need to prepare it for use with our chosen model.

We're using the [all-mpnet-base-v2 from Huggingface](https://huggingface.co/sentence-transformers/all-mpnet-base-v2). This is a very common model used for natural language processing and similarity search.

To make our content usable with this model, we need to segment our code into chunks.

Models will often have a character or a token limit. `allmpnet-base-v2` has a limit of 384 characters, truncating any characters more than that.

We want to make sure that we get AS COMPLETE of a thought as possible. That is to say a complete thought split into two segments will likely both be detected vs having a truncated thought that may include irrelevant content.

We're going to split our content into tweet-like segments of approximately 300 characters with a buffer around 20 characters. We'll also split on phrase boundaries like punctuation or newlines.

**This is a choice**. You can experiment with the parameters to fit your needs. You can also look at other models that have 

We need to read the contents of our text file.

LangChain is a great way to wrap around the work that we're doing in this.

LangChain gives us the ability to select our [embeddings](https://python.langchain.com/docs/integrations/text_embedding/). It also gives us an interface to perform a [recursive split by character](https://python.langchain.com/docs/modules/data_connection/document_transformers/recursive_text_splitter/).

## Let's Make it Happen

Let's create a function that:
- reads the file
- parses the metadata from the file
- splits the content into separate documents with unique identifiers

In [ ]:
import logging
import pathlib
import os

import json
import arrow
import frontmatter
import psycopg
import pprint

from dotenv import load_dotenv
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from psycopg.rows import dict_row

load_dotenv()

# Connection parameters
_CONNECTION_STRING = os.getenv("AIVEN_POSTGRES_SERVICE_URI")
conn = psycopg.connect(_CONNECTION_STRING)

fmt = r"MMMM[\s+]D[\w+,\s+]YYYY"

# define splitter
splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=20,
    separators=[".", "!", "?", "\n"],
    keep_separator="end",
)

# define embeddings. These options are all the defaults and not explicitly needed.
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-mpnet-base-v2",
    model_kwargs={"device": "cpu"},
    encode_kwargs={"normalize_embeddings": False},
)


def load_data(file: pathlib.Path):
    """Chunk data, create embeddings, and index in Postgres."""
    frontmatter_post = frontmatter.loads(
        file.read_text()
    )  # loads the metadata from the file
    base_data = {
        "title": frontmatter_post["title"],
        "show": "Conduit",
        "network": "Relay",
        "network_url": "https://relay.fm",
        "description": frontmatter_post["description"],
        "url": frontmatter_post["url"],
        "pub_date": arrow.get(frontmatter_post["pub_date"], fmt).date().isoformat(),
    }

    try:
        with conn.cursor() as cur:

            transcription_sql = (
                "INSERT INTO transcriptions2 (title, content, meta) VALUES (%s, %s, %s)"
            )
            cur.execute(
                transcription_sql,
                (
                    frontmatter_post["title"],
                    frontmatter_post.content,
                    json.dumps(base_data),
                ),
            )

            insert_query = """
                    INSERT INTO quotes2 (content, embedding, transcription_title)
                    VALUES (%s, %s, %s)
            """

            post_chunks = splitter.create_documents([frontmatter_post.content])
            #
            # Batch embedding generation
            chunk_contents = [chunk.page_content for chunk in post_chunks]
            chunk_embeddings = embeddings.embed_documents(chunk_contents)

            # Prepare batch insert data
            docs = [
                (
                    post_chunk.page_content,
                    embedding,
                    frontmatter_post["title"],
                )
                for post_chunk, embedding in zip(post_chunks, chunk_embeddings)
            ]

            print(f"{file.name} - {len(docs)} chunks")
            cur.executemany(insert_query, docs)
            conn.commit()

    except Exception as e:
        conn.rollback()
        logging.error("Error proccessing %s: %s" % (file.name, e))

Now that we have our function, we'll pass that function into our opensearch bulk function. This will allow us to ingest the documents one at a time, making it easier to restart in the event of an error.

In [ ]:
directory = pathlib.Path("../transcripts")

for file in directory.iterdir():
    load_data(file)


Let's view the transcription item and a few of the quotes.

In [ ]:
with conn.cursor(row_factory=dict_row) as cur:
    cur.execute("SELECT * from transcriptions;")
    result = cur.fetchone()
    pprint.pprint(f"result:{result['title']}, {result['meta']}") 
    cur.execute("SELECT * from quotes LIMIT 5;")
    pprint.pprint(cur.fetchall())

**EXTRA** - If you want run this on all of the files. Change the `directory` path in the block above to `../transcripts_complete` and run the block above again.

In this notebook we did a lot! We chunked. We Generated Embeddings. We also added our example document to our OpenSearch index.

In the next Notebook, we'll look at what we can do now that our transcripts have embeddings and we can interact with our data in OpenSearch and implement it in our RAG pattern using LangChain.

[![Implement our RAG Patter](https://img.shields.io/badge/3-Implement%20in%20RAG-153a5a?style=for-the-badge&labelColor=ec6147)](3-implement-in-rag.ipynb)
